In [18]:
import os
import csv
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

from utils import savefig

plt.rcParams['font.size'] = 14

count accuracy of each hyperparam setting

In [8]:
for j, seqlen in enumerate([4,8,12,16]):
    exp_dir = Path("./experiments/RL/Noise/NBack/figures/ValueMemoryGRU".format(seqlen))
    for noise in ["0", "02", "04", "06", "08", "1"]:
        exp_name = "setup_seq{}_noise{}".format(seqlen, noise)
        accuracy = 0.0
        forward_asymmetry = 0.0
        temporal_factor = 0.0
        cnt = 0
        for i in range(20):
            if not os.path.exists(exp_dir / exp_name / str(i) / "contiguity_effect.csv"):
                continue
            with open(exp_dir / exp_name / str(i) / "contiguity_effect.csv") as f:
                reader = csv.reader(f)
                for row in reader:
                    if float(row[0]) >= 0.5:
                        cnt += 1
                        accuracy += float(row[0])
                        forward_asymmetry += float(row[1])
                        temporal_factor += float(row[2])
        if cnt == 0:
            cnt = 1
        print(exp_name, cnt, np.round(accuracy / cnt, 4), np.round(forward_asymmetry / cnt, 4), np.round(temporal_factor / cnt, 4))
    print()
    print("#########################")


setup_seq4_noise0 20 0.9875 0.784 0.3851
setup_seq4_noise02 20 0.9907 0.8483 0.4278
setup_seq4_noise04 20 0.9913 0.9386 0.4514
setup_seq4_noise06 20 0.9683 0.997 0.4955
setup_seq4_noise08 20 0.8263 0.9796 0.461
setup_seq4_noise1 20 0.827 0.9501 0.3878

#########################
setup_seq8_noise0 20 0.9461 0.7282 0.6317
setup_seq8_noise02 19 0.9652 0.7767 0.6411
setup_seq8_noise04 20 0.9733 0.8383 0.6966
setup_seq8_noise06 20 0.9363 0.8885 0.682
setup_seq8_noise08 20 0.9479 0.9297 0.7098
setup_seq8_noise1 20 0.9867 0.9803 0.7287

#########################
setup_seq12_noise0 20 0.9118 0.6629 0.6704
setup_seq12_noise02 19 0.8932 0.6848 0.6296
setup_seq12_noise04 20 0.8981 0.7057 0.6505
setup_seq12_noise06 20 0.8718 0.794 0.6418
setup_seq12_noise08 20 0.8558 0.8209 0.61
setup_seq12_noise1 20 0.9529 0.9562 0.7787

#########################
setup_seq16_noise0 19 0.8507 0.5136 0.5772
setup_seq16_noise02 18 0.8139 0.4616 0.5217
setup_seq16_noise04 19 0.8985 0.5068 0.6745
setup_seq16_noise06 20

plot performance, FA, TF, proportion of index encoding of different seqlen and noise

In [19]:
def plot_bar(data, std, title, save_path):
    plt.figure(figsize=(6, 4), dpi=180)

    x = np.arange(0, 1.2, 0.2)
    plt.bar(x, data, yerr=std, width=0.1)
    plt.xticks(x, ["0", "0.2", "0.4", "0.6", "0.8", "1"])

    plt.xlabel("Noise proportion")
    plt.ylabel(title)

    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    plt.tight_layout()

    savefig(save_path, title)

def plot_matrix(data, title, save_path, xticks=["0", "0.2", "0.4", "0.6", "0.8", "1"], yticks=["0", "0.3", "0.6", "0.9"],
                xlabel="Noise proportion", ylabel="Gamma", minv=None, maxv=None):
    assert len(xticks) == data.shape[1]
    assert len(yticks) == data.shape[0]

    plt.figure(figsize=(6, 4), dpi=180)

    plt.imshow(data, cmap='viridis', aspect='auto', norm=Normalize(vmin=minv, vmax=maxv))
    plt.colorbar()
    # cb.set_clim(minv, maxv)
    plt.xticks(np.arange(len(xticks)), xticks)
    plt.yticks(np.arange(len(yticks)), yticks)

    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)

    plt.tight_layout()

    savefig(save_path, title)

In [14]:
seqlen = [4, 8, 12, 16]
noise = [0, 0.2, 0.4, 0.6, 0.8, 1]

In [28]:
threshold = [0.9, 0.8, 0.7, 0.6]

# all models with performance > 70

accuracy = np.zeros((4, 6))
forward_asymmetry = np.zeros((4, 6))
temporal_factor = np.zeros((4, 6))
index_coding_prop = np.zeros((4, 6))
cnt = np.zeros((4, 6))


for i, s in enumerate(seqlen):
    for j, n in enumerate(noise):
        exp_path = Path("./experiments/RL/Noise/Gamma09/figures/ValueMemoryGRU/setup_seq{}_noise{}".format(s, str(n).replace(".", "")))
        for k in range(100):
            if not os.path.exists(exp_path / str(k) / "contiguity_effect.csv"):
                continue
            with open(exp_path / str(k) / "contiguity_effect.csv") as f:
                reader = csv.reader(f)
                for row in reader:
                    if float(row[0]) >= 0.7:
                        cnt[i, j] += 1
                        accuracy[i, j] += float(row[0])
                        forward_asymmetry[i, j] += float(row[1])
                        temporal_factor[i, j] += float(row[2])
                        data = np.load(exp_path / str(k) / "pc_selectivity_recalling.npz", allow_pickle=True)
                        if data['selectivity'][1][-1] >= threshold[i]:
                            index_coding_prop[i, j] += 1.0
print(cnt)
cnt[cnt==0] = 1
accuracy /= cnt
forward_asymmetry /= cnt
temporal_factor /= cnt
index_coding_prop /= cnt
print(index_coding_prop)

xticks = ["0", "0.2", "0.4", "0.6", "0.8", "1"]
yticks = [4, 8, 12, 16]
xlabel = "noise proportion"
ylabel = "sequence length"
figpath = Path("./figures/strategy/perf70")

plot_matrix(accuracy, "accuracy", figpath, xticks=xticks, yticks=yticks, xlabel=xlabel, ylabel=ylabel, minv=0.7, maxv=1.0)
plot_matrix(forward_asymmetry, "forward asymmetry", figpath, xticks=xticks, yticks=yticks, xlabel=xlabel, ylabel=ylabel, minv=0.0, maxv=1.0)
plot_matrix(temporal_factor, "temporal factor", figpath, xticks=xticks, yticks=yticks, xlabel=xlabel, ylabel=ylabel, minv=0.3, maxv=0.8)
plot_matrix(index_coding_prop, "index coding proportion", figpath, xticks=xticks, yticks=yticks, xlabel=xlabel, ylabel=ylabel, minv=0.0, maxv=1.0)


[[20. 20. 20. 20. 20. 20.]
 [20. 20. 20. 20. 20. 20.]
 [20. 20. 20. 20. 20. 20.]
 [20. 20. 20. 20. 20. 18.]]
[[1.   1.   1.   0.9  1.   1.  ]
 [0.   0.   0.   0.   0.   0.85]
 [0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.  ]]


In [25]:
threshold = 0.7

# top 20% performing models

accuracy = np.zeros((4, 6))
forward_asymmetry = np.zeros((4, 6))
temporal_factor = np.zeros((4, 6))
index_coding_prop = np.zeros((4, 6))


for i, s in enumerate(seqlen):
    for j, n in enumerate(noise):
        acc, fa, tf, ic = [], [], [], []

        exp_path = Path("./experiments/RL/Noise/Seq{}/figures/ValueMemoryGRU/setup_{}noise{}_gamma{}".format(s, seq_setup_str[i], str(n).replace(".", ""), gamma[i][j]))
        for k in range(100):
            if not os.path.exists(exp_path / str(k) / "contiguity_effect.csv"):
                continue
            with open(exp_path / str(k) / "contiguity_effect.csv") as f:
                reader = csv.reader(f)
                for row in reader:
                    if float(row[0]) >= 0.7:
                        acc.append(float(row[0]))
                        fa.append(float(row[1]))
                        tf.append(float(row[2]))
                        data = np.load(exp_path / str(k) / "pc_selectivity_recalling.npz", allow_pickle=True)
                        if data['selectivity'][1][-1] >= threshold:
                            ic.append(1.0)
                        else:
                            ic.append(0.0)
        if len(acc) == 0:
            continue
        acc = np.array(acc)
        fa = np.array(fa)
        tf = np.array(tf)
        ic = np.array(ic)
        top_acc_idx = np.argsort(acc)[-int(len(acc) * 0.2):]
        accuracy[i, j] = np.mean(acc[top_acc_idx])
        forward_asymmetry[i, j] = np.mean(fa[top_acc_idx])
        temporal_factor[i, j] = np.mean(tf[top_acc_idx])
        index_coding_prop[i, j] = np.mean(ic[top_acc_idx])


xticks = ["0", "0.2", "0.4", "0.6", "0.8", "1"]
yticks = [4, 8, 12, 16]
xlabel = "noise proportion"
ylabel = "sequence length"
figpath = Path("./figures/strategy/toppf20")

plot_matrix(accuracy, "accuracy", figpath, xticks=xticks, yticks=yticks, xlabel=xlabel, ylabel=ylabel)
plot_matrix(forward_asymmetry, "forward asymmetry", figpath, xticks=xticks, yticks=yticks, xlabel=xlabel, ylabel=ylabel)
plot_matrix(temporal_factor, "temporal factor", figpath, xticks=xticks, yticks=yticks, xlabel=xlabel, ylabel=ylabel)
plot_matrix(index_coding_prop, "index coding proportion", figpath, xticks=xticks, yticks=yticks, xlabel=xlabel, ylabel=ylabel)


same thing for n-back models

In [29]:
threshold = [0.9, 0.8, 0.7, 0.6]

# all models with performance > 70

accuracy = np.zeros((4, 6))
forward_asymmetry = np.zeros((4, 6))
temporal_factor = np.zeros((4, 6))
index_coding_prop = np.zeros((4, 6))
cnt = np.zeros((4, 6))


for i, s in enumerate(seqlen):
    for j, n in enumerate(noise):
        exp_path = Path("./experiments/RL/Noise/NBack/figures/ValueMemoryGRU/setup_seq{}_noise{}".format(s, str(n).replace(".", "")))
        for k in range(20):
            if not os.path.exists(exp_path / str(k) / "contiguity_effect.csv"):
                continue
            with open(exp_path / str(k) / "contiguity_effect.csv") as f:
                reader = csv.reader(f)
                for row in reader:
                    if float(row[0]) >= 0.7:
                        cnt[i, j] += 1
                        accuracy[i, j] += float(row[0])
                        forward_asymmetry[i, j] += float(row[1])
                        temporal_factor[i, j] += float(row[2])
                        data = np.load(exp_path / str(k) / "pc_selectivity_recalling.npz", allow_pickle=True)
                        if data['selectivity'][1][-1] >= threshold[i]:
                            index_coding_prop[i, j] += 1.0
print(cnt)
cnt[cnt==0] = 1
accuracy /= cnt
forward_asymmetry /= cnt
temporal_factor /= cnt
index_coding_prop /= cnt
print(index_coding_prop)

xticks = ["0", "0.2", "0.4", "0.6", "0.8", "1"]
yticks = [4, 8, 12, 16]
xlabel = "noise proportion"
ylabel = "sequence length"
figpath = Path("./figures/strategy/nback")

plot_matrix(accuracy, "accuracy", figpath, xticks=xticks, yticks=yticks, xlabel=xlabel, ylabel=ylabel, minv=0.7, maxv=1.0)
plot_matrix(forward_asymmetry, "forward asymmetry", figpath, xticks=xticks, yticks=yticks, xlabel=xlabel, ylabel=ylabel, minv=0.0, maxv=1.0)
plot_matrix(temporal_factor, "temporal factor", figpath, xticks=xticks, yticks=yticks, xlabel=xlabel, ylabel=ylabel, minv=0.3, maxv=0.8)
plot_matrix(index_coding_prop, "index coding proportion", figpath, xticks=xticks, yticks=yticks, xlabel=xlabel, ylabel=ylabel, minv=0.0, maxv=1.0)


[[20. 20. 20. 20. 20. 20.]
 [20. 19. 20. 20. 20. 20.]
 [19. 17. 20. 19. 20. 20.]
 [16. 14. 19. 16. 15. 19.]]
[[0.95       0.95       1.         1.         1.         1.        ]
 [0.95       1.         0.9        0.85       0.8        0.95      ]
 [0.63157895 0.64705882 0.55       0.57894737 0.25       0.9       ]
 [0.125      0.         0.15789474 0.         0.2        0.89473684]]
